In [1]:
from imutils.perspective import four_point_transform
from skimage.segmentation import clear_border
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import img_to_array


import imutils
import cv2 as cv
from PIL import Image

In [2]:
LEARNING_RATE = 0.001


In [3]:
def find_puzzle(image, debug=False):
	gray = cv.cvtColor(src=image, code=cv.COLOR_BGR2GRAY)
	blurred = cv.GaussianBlur(src=gray, ksize=(7,7), sigmaX=3)

	thresh = cv.adaptiveThreshold(src=blurred, maxValue=255, 
			       adaptiveMethod=cv.ADAPTIVE_THRESH_GAUSSIAN_C, 
				   thresholdType=cv.THRESH_BINARY, blockSize=11, C=2)
	thresh = cv.bitwise_not(src=thresh)  # Contours become white

	contours = cv.findContours(image=thresh.copy(), mode=cv.RETR_EXTERNAL, method=cv.CHAIN_APPROX_SIMPLE)
	contours = imutils.grab_contours(cnts=contours)
	contours = sorted(contours, key=cv.contourArea, reverse=True)

	puzzle_contour = None
	for contour in contours:
		perimeter = cv.arcLength(curve=contour, closed=True)
		estimate = cv.approxPolyDP(curve=contour, epsilon=0.02*perimeter, closed=True)

		if len(estimate) == 4:
			puzzle_contour = estimate
			break

	if puzzle_contour is None:
		raise Exception(("Could not find Sudoku puzzle outline. "
			"Try debugging your thresholding and contour steps."))

	puzzle = four_point_transform(image=image, pts=puzzle_contour.reshape(4, 2))
	warped = four_point_transform(image=gray,  pts=puzzle_contour.reshape(4, 2))
	
	if debug:
		cv.imshow("Puzzle Input", image)
		cv.waitKey(0)

		cv.imshow("Puzzle Gray", gray)
		cv.waitKey(0)

		cv.imshow("Puzzle Blurred", blurred)
		cv.waitKey(0)

		cv.imshow("Puzzle Thresh", thresh)
		cv.waitKey(0)

		output = image.copy()
		cv.drawContours(image=output, contours=[puzzle_contour], contourIdx=-1, color=(255, 0, 0), thickness=3)
		cv.imshow("Puzzle Outline", output)
		cv.waitKey(0)

		cv.imshow("Puzzle Transform", puzzle)
		cv.waitKey(0)

		cv.imshow("Puzzle Warped", warped)
		cv.waitKey(0)
	return (puzzle, warped)

In [4]:
def extract_digit(cell, debug=False):
	_, thresh = cv.threshold(src=cell, thresh=0, maxval=255, type= cv.THRESH_BINARY_INV | cv.THRESH_OTSU)
	thresh = clear_border(thresh)

	contours = cv.findContours(image=thresh.copy(), mode=cv.RETR_EXTERNAL, method=cv.CHAIN_APPROX_SIMPLE)
	contours = imutils.grab_contours(cnts=contours)

	# Empty cell
	if len(contours) == 0:
		return None

	# Get largest contour
	mask = np.zeros(thresh.shape, dtype="uint8")
	contour = max(contours, key=cv.contourArea)
	cv.drawContours(image=mask, contours=[contour], contourIdx=-1, color=255, thickness=-1)

	h, w = thresh.shape
	if cv.countNonZero(mask) / float(w * h) < 0.03:  # Likely just noise --> 0
		return None

	digit = cv.bitwise_and(src1=thresh, src2=thresh, mask=mask)  # Why not just mask

	if debug:
		cv.imshow("Cell Thresh", thresh)
		cv.waitKey(0)
		cv.imshow("Digit", digit)
		cv.waitKey(0)

	return digit

In [5]:

image = cv.imread('../input/picca.webp')
puzzleImage, warped = find_puzzle(image, debug=False)
board = np.zeros((9, 9), dtype="int")
stepX = warped.shape[1] // 9
stepY = warped.shape[0] // 9
cellLocs = []
model = load_model('digit_classifier2.h5', compile=False)
for y in range(0, 9):
	row = []
	for x in range(0, 9):
		startX = x * stepX
		startY = y * stepY
		endX = (x + 1) * stepX
		endY = (y + 1) * stepY
		row.append((startX, startY, endX, endY))

		cell = warped[startY:endY, startX:endX]
		digit = extract_digit(cell, debug=False)
		if digit is not None:
			roi = cv.resize(digit, (28, 28))
			roi = roi.astype("float") / 255.0
			# roi = img_to_array(roi)
			roi = np.expand_dims(roi, axis=0)
			# classify the digit and update the Sudoku board with the
			# prediction
			predictions = model.predict(roi)[0][6]
			pred = model.predict(roi).argmax(axis=1)[0]
			if pred == 8 and predictions > 0.00002:  ## 6 is barely recognized
				pred = 6
			print(pred)
			board[y, x] = pred
	# add the row to our cell locations
	cellLocs.append(row)

1/1 [==============================] - 0s 13ms/step
8
1/1 [==============================] - 0s 16ms/step
1
1/1 [==============================] - 0s 18ms/step
9
1/1 [==============================] - 0s 16ms/step
5
1/1 [==============================] - 0s 15ms/step
8
1/1 [==============================] - 0s 16ms/step
7
1/1 [==============================] - 0s 16ms/step
1
1/1 [==============================] - 0s 16ms/step
4
1/1 [==============================] - 0s 38ms/step
9
1/1 [==============================] - 0s 20ms/step
7
1/1 [==============================] - 0s 15ms/step
6
1/1 [==============================] - 0s 19ms/step
7
1/1 [==============================] - 0s 8ms/step
1
1/1 [==============================] - 0s 14ms/step
2
1/1 [==============================] - 0s 19ms/step
5
1/1 [==============================] - 0s 11ms/step
8
1/1 [==============================] - 0s 17ms/step
6
1/1 [==============================] - 0s 11ms/step
1
1/1 [========================

In [6]:
correct = [[8, 0, 0, 0, 1, 0, 0, 0, 9],
           [0, 5, 0, 8, 0, 7, 0, 1, 0],
           [0, 0, 4, 0, 9, 0, 7, 0, 0],
           [0, 6, 0, 7, 0, 1, 0, 2, 0],
           [5, 0, 8, 0, 6, 0, 1, 0, 7],
           [0, 1, 0, 5, 0, 2, 0, 9, 0],
           [0, 0, 7, 0, 4, 0, 6, 0, 0],
           [0, 8, 0, 3, 0, 9, 0, 4, 0],
           [3, 0, 0, 0, 5, 0, 0, 0, 8]]
print(board)
print(np.array_equal(np.matrix(board), np.matrix(correct)))

[[8 0 0 0 1 0 0 0 9]
 [0 5 0 8 0 7 0 1 0]
 [0 0 4 0 9 0 7 0 0]
 [0 6 0 7 0 1 0 2 0]
 [5 0 8 0 6 0 1 0 7]
 [0 1 0 5 0 2 0 9 0]
 [0 0 7 0 4 0 6 0 0]
 [0 8 0 3 0 9 0 4 0]
 [3 0 0 0 5 0 0 0 8]]
True


In [7]:
import copy

def solve(configuration):
    board = copy.deepcopy(configuration)

    def solve_h():
        def possible_placement(r, c, n):
            if (any(board[r][i]==n for i in range(9))
                or any(board[i][c]==n for i in range(9))
                or any(board[i][j]==n for i in range(3*(r//3), 3*(r//3)+3) 
                                        for j in range(3*(c//3), 3*(c//3)+3))):
                return False
            return True
        
        for r in range(9):
            for c in range(9):
                if board[r][c] != 0:
                    continue
                for n in (1,2,3,4,5,6,7,8,9):
                    if possible_placement(r, c, n):
                        board[r][c] = n
                        if solve_h():
                            return True
                        board[r][c] = 0
                return False
        return True
    solve_h()
    
    return board

solution = solve(board)

In [8]:
# loop over the cell locations and board
for r, (cellRow, boardRow) in enumerate(zip(cellLocs, solution)):
	# loop over individual cell in the row
	for c, (box, digit) in enumerate(zip(cellRow, boardRow)):
		# unpack the cell coordinates
		startX, startY, endX, endY = box
		# compute the coordinates of where the digit will be drawn
		# on the output puzzle image
		textX = int((endX - startX) * 0.33)
		textY = int((endY - startY) * -0.2)
		textX += startX
		textY += endY
		# draw the result digit on the Sudoku puzzle image
		if not board[r, c]:
			cv.putText(puzzleImage, str(digit), (textX, textY), cv.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
# show the output image
cv.imshow("Sudoku Result", puzzleImage)
cv.waitKey(0)

-1